# Group comparison 3T and 7T epilepsy  

Surface-based comparisons  
- vertex-wise T-test : are controls and pts different
    - Use brainstat
- vertex-wise effect size : how big are the distances between the vertex differences?
    - Use own function


For figures: 
- Visualize effect size on a brain masked for significant p-values 

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import pickle
import datetime
import brainstat as bstat

In [2]:
# specify root directories
MICs = {
    "name": "MICs",
    "dir_root": "/data/mica3/BIDS_MICs",
    "dir_mp": "micapipe_v0.2.0",
    "dir_hu": "hippunfold_v1.3.0/hippunfold",
    "study": "3T",
    "ID_ctrl" : ["HC"],
    "ID_Pt" : ["PX"]
    }

PNI = {
    "name": "PNI",
    "dir_root": "/data/mica3/BIDS_PNI",
    "dir_mp": "micapipe_v0.2.0",
    "dir_hu": "hippunfold_v1.3.0/hippunfold",
    "study": "7T",
    "ID_col" : ["PNC", "Pilot"], # column for ID in demographics file
    }

studies = [MICs, PNI]

demographics = {
    "pth" : "/host/verges/tank/data/daniel/3T7T/z/data/pt/demo_22May2025.csv",
    # column names:
    "ID_7T" : "PNI_ID", 
    "ID_3T" : "MICS_ID",
    "SES" : "SES",
    "date": "Date",
    "grp" : "grp_detailed" # col name for participant grouping variable of interest
}

px_grps = { # specify patient group labels to compare to controls
    'all' : ['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'],
    'TLE' : ['TLE_L', 'TLE_R', 'TLE_U', 'mTLE_R', 'mTLE_L'],
    'TLE_L': ['TLE_L', 'mTLE_L'],
    'TLE_R': ['TLE_R', 'mTLE_R'],
    'FCD' : ['FLE_R', 'FLE_L'],
    'MFCL' : ['MFCL', 'MFCL_bTLE'],
    'UKN' : ['UKN_L', 'UKN_U']
}

ctrl_grp = {'ctrl' : ['CTRL']}

groups = {**px_grps, **ctrl_grp} # Combine all patient and control groups into a single dictionary

surfaces = ["fsLR-5k"]
labels = ["pial", "white", "midThick"]

In [3]:
# retrieve surfaces from pt of interest
def load_surf(study, IDss):
    """
    Get the surface data for a given group.

    inputs:
    study: dictionary item with keys 'name', 'dir_root', 'study'
    IDs: pd.dataframe woth cols IDs and SES indicating all participants IDs to extract surfaces for

    outputs:
    surfs: pd.dataframe with vertices in rows and unique ID_SES in columns
    """
    
    import nibabel as nib
    
    # get the list of patients in the group
    pt_list = bstat.get_subjects(study, pt_grp)
    
    # get the surface data for each patient
    surf_data = {}
    for pt in pt_list:
        surf_data[pt] = {}
        for surf in surfaces:
            surf_data[pt][surf] = bstat.get_surface_data(study, pt, surf)
    
    return surf_data

In [4]:
def chk_pth(pth):
    """
    Check if the path exists and is a file.
    
    inputs:
        pth: path to check
    
    outputs:
        True if the path exists and is a file, False otherwise
    """
    
    import os
    
    if os.path.exists(pth) and os.path.isfile(pth):
        return True
    else:
        return False

In [5]:
def mp_mapsPth(dir,sub, ses, hemi, surf, lbl, ft):
    """
    Returns path to micapipe maps for given subject, session, hemisphere, surface, label, and feature.
    """
    return f"{dir}/sub-{sub}_ses-{ses}_hemi-{hemi}_surf-{surf}_label-{lbl}_{ft}.func.gii"

# when working, add to Utils scripts
def get_1pth(root, deriv_fldr, sub, ses, label="midthickness_T1map", surf="fsLR-5k", feature="flair", space="nativepro", hemi="LR", check_pth=True,silence=True):
    """
    Get the path to the surface data for a given subject and session.
    Assumes BIDS format of data storage.

    inputs:
        root: root directory of the study
        deriv_fldr: name of derivative folder containing the surface data
        sub: subject ID (no `sub-` prefix)
        ses: session ID (with leading zero if applicable; no `ses-` prefix)
        surf: surface type and resolution (e.g., fsLR-32k, fsLR-5k)
        label: surface label (e.g., "pial", "white", "midThick")
        space: space of the surface data (e.g., "nativepro", "fsnative")
        hemi: hemisphere to extract (default is "LR" for both left and right hemispheres)

        check_pth: whether to check if the path exists (default is True)
        silence: whether to suppress print statements (default is True)
    outputs:
        path to the surface data files
    """

    # make surf to lower case
    label = label.lower()

    # ensure that surface is well defined
    if label == "pial":
        label = "pial"
    elif label == "white":
        label = "white"
    elif label == "midThick" or label == "midthickness":
        label = "midthickness"
    else:
        raise ValueError("Invalid surface type. Choose from 'pial', 'white', or 'midThick'.")
    
    # construct the path to the surface data file
    
    hemi = hemi.upper()
    if hemi == "LEFT" or hemi == "L":
        hemi = "L"
    elif hemi == "RIGHT" or hemi == "R":
        hemi = "R"
    elif hemi != "LR":
        raise ValueError("Invalid hemisphere. Choose from 'L', 'R', or 'LR'.")

    # handle hippunfold naming convention
    if "micapipe" in deriv_fldr.lower():
        pth = f"{root}/derivatives/{deriv_fldr}/sub-{sub}/ses-{ses}/maps"
        if hemi == "L" or hemi == "R":
            pth = mp_mapsPth(dir=pth, sub=sub, ses=ses, hemi=hemi, surf=surf, lbl=label, ft=feature)
            if not silence: print(f"[get_1pth] Returning paths for both hemispheres ([0]: L, [1]: R)")
            
        else:         
            pth_L = mp_mapsPth(dir=pth, sub=sub, ses=ses, hemi="L", surf=surf, lbl=label, ft=feature)
            pth_R = mp_mapsPth(dir=pth, sub=sub, ses=ses, hemi="R", surf=surf, lbl=label, ft=feature)        
            pth = [pth_L, pth_R]
            if not silence: print(f"[get_1pth] Returning paths for both hemispheres ([0]: L, [1]: R)")
    elif "hippunfold" in deriv_fldr.lower():
        raise ValueError("Hippunfold derivative not yet implemented. Need to create feature maps using hippunfold surfaces.")
        
        # space usually: "T1w"
        # surf usually: "fsLR"
        # label options: "hipp_outer", "hipp_inner", "hipp_midthickness"

        pth = f"{root}/derivatives/{deriv_fldr}/sub-{sub}/ses-{ses}/surf"

        if hemi == "L" or hemi == "R":
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_space-{space}_den-{surf}_label-{label}.surf.gii"
            if not silence: print(f"[surf_pth] Returning hippunfold path for {hemi} hemisphere")
        else:
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_surf-{surf}_label-{label}_{feature}.func.gii"
            pth_L = f"{pth}/sub-{sub}_ses-{ses}_hemi-L_-{surf}_label-{label}.surf.gii"
            pth_R = f"{pth}/sub-{sub}_ses-{ses}_hemi-R_space-{space}_den-{surf}_label-{label}.surf.gii"
            pth = [pth_L, pth_R]
            if not silence: print(f"[surf_pth] Returning hippunfold paths for both hemispheres ([0]: L, [1]: R)")

    else:
        raise ValueError("Invalid derivative folder. Choose from 'micapipe' or 'hippunfold'.")


    if check_pth:
        if isinstance(pth, list):
            for idx, p in enumerate(pth):
                if not chk_pth(p):
                    print(f"\t[get_1pth] FILE NOT FOUND (sub-{sub}_ses-{ses}): {p}")
                    pth[idx] = "ERROR:" + p
        else:
            if not chk_pth(pth):
                print(f"\t[get_1pth] FILE NOT FOUND (sub-{sub}_ses-{ses}): {pth}")
                pth = "ERROR:" + pth
    
    return pth   

def get_Npths(demographics, study, groups, feature="FA", derivative="micapipe", label="midthickness", hemi="LR", space="nativepro", surf="fsLR-5k"):
    """
    Get path to surface files for individual groups


    Input:
    demographics: dict  regarding demographics file. 
        Required keys: 
            'pth'
            'ID_7T'
            'ID_3T'
            'SES'
            'date'
            'grp'
    study: dict  regarding study.
        Required keys: 
            'name'
            'dir_root'
            'study'
            'dir_mp'
            'dir_hu'
    groups: dict    of groups to extract surfaces for. 
        Each key should be a group name, and the value should be a list of labels in the 'grp' column of demographics file assigned to that group.
    label: str  surface label to extract
    hemi: str  hemisphere to extract. Default is "LR" for both left and right hemispheres.
    space: str  space of the surface data. Default is "nativepro".
    surf: str  surface type and resolution. Default is "fsLR-5k".
    """
    import pandas as pd

    demo = pd.read_csv(demographics['pth'], dtype=str)
    
    out = []

    if derivative == "hippunfold":
        deriv_fldr = study['dir_hu']
    elif derivative == "micapipe":
        deriv_fldr = study['dir_mp']
    else:
        deriv_fldr = study['dir_mp']
        print(f"[get_Npths] WARNING: derivative not recognized. Defaulting to micapipe.")


    for grp_name, grp_labels in groups.items():
        print(f"{study['name']} {grp_name} ({grp_labels})")

        # get IDs for this group
        ids = demo.loc[
            (demo[demographics['grp']].isin(grp_labels)) &
            (demo['study'] == study['study']),
            [ID_col, demographics['SES'], 'study', 'Date']
        ].copy()

        for i, row in ids.iterrows():
            ID = row[ID_col]
            SES = row[demographics['SES']]
            date = row[demographics['date']]
            #print(f"\tsub-{ID}_ses-{SES}")
            pth = get_1pth(root=study['dir_root'], deriv_fldr=deriv_fldr, sub=ID, ses=SES, label=label, surf=surf, feature=feature, space=space, hemi=hemi)
            # add this pth to the dataframe
            if isinstance(pth, list):
                ids.loc[i, f'pth_L'] = pth[0]
                ids.loc[i, f'pth_R'] = pth[1]
            else:
                ids.loc[i, f'pth_{hemi}'] = pth 
        # if paths are duplicated, then keep only one of those rows
        if hemi == "LR":
            ids = ids.drop_duplicates(subset=[f'pth_L', f'pth_R'])
        else:
            ids = ids.drop_duplicates(subset=[f'pth_{hemi}'])

        # create dictionary item for each group, add to output list
        out.append({
            'study': study['name'],
            'grp': grp_name,
            'grp_labels': grp_labels,
            'label': label,
            'feature': feature,
            'map_pths': ids
        })

    return out


In [ ]:
def clean_pths(dl, method="newest"):
    """
    Keeps only one session per ID
    input:
        dl (for dictionary list): List of dictionary items (e.g. outputs from get_Npths). 
            These dict should contain a df under the key 'map_pths'
        method: method to use for choosing session.
            "newest": use most recent session
            "oldest": use oldest session in the list
            {number}: session code to use (e.g. '01' or 'a1' etc)

    output:
        dl: List of dictionary items with cleaned dataframes

    """
    dl_out = []
    # iterate thorugh each dictionary item in the list
    for i, d in enumerate(dl):
        dl_out.append(d.copy()) # create a copy of the dictionary item
        #print(i)
        # get the dataframe
        
        if d['study'] == "PNI": ID_col = "PNI_ID"
        else: ID_col = "MICS_ID"
        #print(ID_col)

        df = d['map_pths']
        print(f"[clean_pths] {d['study']} {d['grp']}: {df.shape}, num unique IDs: {df[ID_col].nunique()}")
        
        if df.empty: # check if the dataframe is empty
            print(f"\t[clean_pths] WARNING: Empty dataframe for {d['study']} {d['grp']}")
            continue
        else:
            df_clean = ses_clean(df, ID_col, method=method, silent=False)
            #dl[i]['map_pths'] = df_clean
    
        # add df to dictionary item
        dl_out[i]['map_pths'] = df_clean

    return dl


def ses_clean(df, ID_col, method="newest", silent=True):
    """
    Choose the session to use for each subject.
        If subject has multiple sessions with map path should only be using one of these sessions.

    inputs:
        df: pd.dataframe with columns for subject ID, session, date and map_paths
            Assumes map path is missing if either : map_pth
        method: method to use for choosing session. 
            "newest": use most recent session
            "oldest": use oldest session in the list
            {number}: session code to use (e.g. '01' or 'a1' etc)
    """
    
    import pandas as pd
    import datetime

    # check if the dataframe is empty
    if df.empty:
        print(f"\t[ses_clean] WARNING: Empty dataframe for {d['study']} {d['grp']}")
        return

    if not silent: print(f"[ses_clean] Choosing session according to method: {method}")
    # Find path cols
    path_cols = [col for col in df.columns if col.startswith('pth_') or col.startswith('surf_') or col.startswith('map_')]
    #if not silent: print (f"\t[ses_clean] Path columns: {path_cols}")

    # remove rows with missing map paths
    valid = df[path_cols].apply(lambda row: any((isinstance(x, str) and x and not x.startswith("ERROR")) for x in row), axis=1)
    if not silent: print(f"\tMISSING PATHS: {df.shape[0] - valid.sum()} removed; Change in unique IDs: {df[ID_col].nunique() - df[valid][ID_col].nunique()}")

    # Filter to valid rows only
    df_valid = df[valid].copy()

    # Find repeated IDs (i.e., subjects with multiple sessions)
    # Identify IDs that appear more than once (i.e., repeated IDs)
    repeated_ids = df_valid[df_valid.duplicated(subset=ID_col, keep=False)][ID_col].unique()
    if not silent:
        if len(repeated_ids) > 0:
            print(f"\tIDs with multiple valid sessions: {repeated_ids}")
        else:
            print(f"\tNo repeated IDs found")

    rows_to_remove = []
    
    # Convert 'Date' column to datetime for comparison
    df_valid['Date_dt'] = pd.to_datetime(df_valid['Date'], format='%d.%m.%Y', errors='coerce')
    today = pd.to_datetime('today').normalize()

    if len(repeated_ids) > 0:
        if method == "newest":
            for id in repeated_ids:
                sub_df = df_valid[df_valid[ID_col] == id]
                if sub_df.shape[0] > 1:
                    # Keep only the newest (most recent) session
                    idx_to_keep = sub_df['Date_dt'].idxmax()
                    idx_to_remove = sub_df.index.difference([idx_to_keep])
                    rows_to_remove.extend(idx_to_remove)
        elif method == "oldest":
            for id in repeated_ids:
                sub_df = df_valid[df_valid[ID_col] == id]
                if sub_df.shape[0] > 1:
                    # Keep only the oldest session
                    idx_to_keep = sub_df['Date_dt'].idxmin()
                    idx_to_remove = sub_df.index.difference([idx_to_keep])
                    rows_to_remove.extend(idx_to_remove)
        else:
            # Assume method is a session code (e.g., '01', 'a1', etc)
            for id in repeated_ids:
                sub_df = df_valid[df_valid[ID_col] == id]
                if sub_df.shape[0] > 1:
                    idx_to_remove = sub_df[sub_df['SES'] != method].index
                    rows_to_remove.extend(idx_to_remove)

    # Remove the rows marked for removal
    df_clean = df_valid.drop(rows_to_remove)
    #if not silent: print(df_clean[[ID_col, 'SES']].sort_values(by=ID_col))

    # if num rows =/= to num unique IDs then write warning
    if df_clean.shape[0] != df_clean[ID_col].nunique():
        print(f"[ses_clean] WARNING: Number of rows ({df_clean.shape[0]}) not equal to num unique IDs ({df_clean[ID_col].nunique()})")
        print(f"\t multiple ses: {df_valid[df_valid.duplicated(subset=ID_col, keep=False)][ID_col].unique()}")

    if not silent: 
        print(f"\t {df_valid.shape[0] - df_clean.shape[0]} rows removed, Change in unique IDs: {df_valid[ID_col].nunique() - df_clean[ID_col].nunique()}")
        print(f"\t{df_clean.shape[0]} rows remaining")

    return df_clean

def get_finalSES(dl, save_pth=None):
    """
    From a list of dictionary items, create a DF with sessions retained for each participant and each feature 

    input:
        dl: List of dictionary items with cleaned dataframes
        save_pth: path to save the dataframe to. If None, do not save.

    output:
        df: pd.dataframe with columns for subject ID, session_feature, date and map_paths
            Assumes map path is missing if either : map_pth
    """

    # can create a seperate function that collects sessions for different features
    for i, d in enumerate(dl):
        feature = d['feature']
        label = d['label']
        grp = d['grp']
        study = d['study']

        # get the dataframe
        df = d['map_pths']

        # copy ID, ses
        ID_col = df.columns[0]
        SES_col = df.columns['SES']

        # create new column for session_feature
        df[f'ses_{label}-{feature}'] = df[SES_col]
        df[grp] = grp

        return df[['study', ID_col, f'ses_{label}-{feature}']]      

In [7]:
demo = pd.read_csv(demographics['pth'], dtype=str)
demo[["MICS_ID", "PNI_ID", "study", "SES", "grp", "grp_detailed"]]

,MICS_ID,PNI_ID,study,SES,grp,grp_detailed
0,HC129,Pilot013,7T,05,CTRL,CTRL
1,HC082,PNC003,7T,01,CTRL,CTRL
2,HC082,PNC003,7T,02,CTRL,CTRL
3,HC082,PNC003,7T,03,CTRL,CTRL
4,HC082,PNC003,7T,04,CTRL,CTRL
...,...,...,...,...,...,...
116,HC130,PNC026,3T,02,CTRL,CTRL
117,HC083,PNC011,3T,02,CTRL,CTRL
118,PX215,PNE020,3T,01,UKN,UKN_U
119,PX216,PNE021,3T,01,TLE,TLE_R


In [8]:
label = "midthickness"
feature = "FA"

map_pths = []
if 'surf_dfs' not in locals():
    surf_dfs = {}

for study in studies:
    if study['study'] == "3T":
        ID_col = demographics['ID_3T']
    elif study['study'] == "7T":
        ID_col = demographics['ID_7T']

    map_pths.extend(get_Npths(demographics, study, groups, feature, derivative="micapipe"))

# save
save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/paths"
date = datetime.datetime.now().strftime("%d%b%Y")
#with open(f'{save_pth}/map_paths_{label}-{feature}_{date}.pkl', 'wb') as f:    pickle.dump(map_pths, f)


MICs all (['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'])
	[get_1pth] FILE NOT FOUND (sub-PX071_ses-03): /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0/sub-PX071/ses-03/maps/sub-PX071_ses-03_hemi-L_surf-fsLR-5k_label-midthickness_FA.func.gii
	[get_1pth] FILE NOT FOUND (sub-PX071_ses-03): /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0/sub-PX071/ses-03/maps/sub-PX071_ses-03_hemi-R_surf-fsLR-5k_label-midthickness_FA.func.gii
	[get_1pth] FILE NOT FOUND (sub-PX216_ses-01): /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0/sub-PX216/ses-01/maps/sub-PX216_ses-01_hemi-L_surf-fsLR-5k_label-midthickness_FA.func.gii
	[get_1pth] FILE NOT FOUND (sub-PX216_ses-01): /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0/sub-PX216/ses-01/maps/sub-PX216_ses-01_hemi-R_surf-fsLR-5k_label-midthickness_FA.func.gii
MICs TLE (['TLE_L', 'TLE_R', 'TLE_U', 'mTLE_R', 'mTLE_L'])
	[get_1pth] FILE NOT FOUND (sub-PX071_ses-03): /data/mica3/BIDS_MICs/derivati

PNI all (['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'])
	[get_1pth] FILE NOT FOUND (sub-PNE002_ses-a1): /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE002/ses-a1/maps/sub-PNE002_ses-a1_hemi-L_surf-fsLR-5k_label-midthickness_FA.func.gii
	[get_1pth] FILE NOT FOUND (sub-PNE002_ses-a1): /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE002/ses-a1/maps/sub-PNE002_ses-a1_hemi-R_surf-fsLR-5k_label-midthickness_FA.func.gii
	[get_1pth] FILE NOT FOUND (sub-PNE003_ses-a1): /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE003/ses-a1/maps/sub-PNE003_ses-a1_hemi-L_surf-fsLR-5k_label-midthickness_FA.func.gii
	[get_1pth] FILE NOT FOUND (sub-PNE003_ses-a1): /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE003/ses-a1/maps/sub-PNE003_ses-a1_hemi-R_surf-fsLR-5k_label-midthickness_FA.func.gii
	[get_1pth] FILE NOT FOUND (sub-PNE004_ses-a1): /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE004/ses-a1/maps/sub-PNE00

In [9]:
# load list of dict items with surface paths

#with open('map_pths.pkl', 'rb') as f:
#    map_pths = pickle.load(f)

In [10]:
# print maps_pths
for i, d in enumerate(map_pths):
    print(f"{i}: {d['study']} {d['grp']} ({d['grp_labels']})")
    print(d['map_pths'])
    print()

0: MICs all (['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'])
    MICS_ID SES study        Date  \
80    PX071  01    3T  01.06.2022   
86    PX071  02    3T  17.01.2023   
87    PX071  03    3T  11.04.2023   
90    PX119  01    3T  30.08.2023   
91    PX071  04    3T  26.09.2023   
95    PX137  01    3T  06.03.2024   
96    PX148  01    3T  06.05.2024   
98    PX153  01    3T  28.05.2024   
99    PX158  01    3T  11.06.2024   
103   PX168  01    3T  20.08.2024   
104   PX173  01    3T  28.08.2024   
105   PX174  01    3T  04.09.2024   
106   PX176  01    3T  05.09.2024   
107   PX183  01    3T  16.10.2024   
109   PX189  01    3T  12.11.2024   
110   PX190  01    3T  12.11.2024   
111   PX194  01    3T  26.11.2024   
112   PX198  01    3T  03.12.2024   
113   PX199  01    3T  03.12.2024   
114   PX200  01    3T  03.12.2024   
115   PX204  01    3T  09.01.2025   
118   PX215  01    3T  20.03.2025   
119   PX216  01    3T  26.03.2

In [36]:
# print specific study-group combinations
get_study = "MICs"
get_grp = "ctrl"

for item in map_pths:
    if item['study'] == get_study and item['grp'] == get_grp:
        print(f"{item['study']}-{item['grp']} ({item['grp_labels']})")
        with pd.option_context('display.max_columns', None):
            print(item['map_pths'])
        break

MICs-ctrl (['CTRL'])
    MICS_ID SES study        Date  \
74    HC062  01    3T  25.08.2020   
75    HC062  02    3T  08.09.2020   
76    HC076  01    3T  08.02.2022   
77    HC081  01    3T  26.04.2022   
78    HC082  01    3T  26.04.2022   
79    HC083  01    3T  12.07.2022   
82    HC088  01    3T  19.07.2022   
83    HC062  03    3T  16.08.2022   
84    HC076  02    3T  27.09.2022   
85    HC081  02    3T  22.11.2022   
88    HC082  02    3T  25.05.2023   
89    HC116  01    3T  04.07.2023   
92    HC081  03    3T  24.10.2023   
93    HC088  02    3T  31.10.2023   
94    HC076  03    3T  06.02.2024   
100   HC128  01    3T  02.06.2024   
101   HC129  01    3T  09.07.2024   
102   HC130  01    3T  09.07.2024   
108   HC140  01    3T  23.10.2024   
116   HC130  02    3T  15.01.2025   
117   HC083  02    3T  28.01.2025   
120   HC076  04    3T  17.04.2025   

                                                 pth_L  \
74   /data/mica3/BIDS_MICs/derivatives/micapipe_v0....   
75   ERROR:

In [12]:
# Find path for specific participant
study = "MICS"
sub = "PNC018"
ses = "01"

grp = "ctrl"

entry = next((item for item in map_pths if item['study'] == study and item['grp'] == grp), None)

if entry is not None:
    df = entry['map_pths']
    row = df[(df[ID_col] == sub) & (df['SES'] == ses)]
    if not row.empty:
        print(row['pth_L'].values[0])
    else:
        print(f"No entry found for subject {sub} and session {ses}")
else:
    print(f"No entry found for study {study} and group {grp}")


No entry found for study MICS and group ctrl


In [107]:
test = [map_pths[1], map_pths[7]]
# print study-grp for test
for i, d in enumerate(test):
    print(f"{i}: {d['study']} {d['grp']}")
    print()

0: MICs TLE

1: MICs ctrl



In [115]:
#print(test[0]['map_pths'][[ID_col, 'SES']].sort_values(by=ID_col))
map_pths_clean = clean_pths(test, method="newest")

for i, d in enumerate(map_pths_clean):
    print(f"\nItem {i}:")
    print(f"  Keys: {list(d.keys())}")


[clean_pths] MICs TLE: (13, 6), num unique IDs: 10
[ses_clean] Choosing session according to method: newest
	MISSING PATHS: 2 removed; Change in unique IDs: 1
	IDs with multiple valid sessions: ['PX071']
	Resulting rows: removed: 2, Change in unique IDs: 0
	9 rows remaining
[clean_pths] MICs ctrl: (22, 6), num unique IDs: 11
[ses_clean] Choosing session according to method: newest
	MISSING PATHS: 1 removed; Change in unique IDs: 0
	IDs with multiple valid sessions: ['HC062' 'HC076' 'HC081' 'HC082' 'HC083' 'HC088' 'HC130']
	Resulting rows: removed: 10, Change in unique IDs: 0
	11 rows remaining

Item 0:
  Keys: ['study', 'grp', 'grp_labels', 'label', 'feature', 'map_pths']

Item 1:
  Keys: ['study', 'grp', 'grp_labels', 'label', 'feature', 'map_pths']


In [ ]:
# finalize sessions to use
map_pths_clean = []
# pass map_paths into function that chooses a single session per subject
# call 1) clean_pths
for index, data in enumerate(map_pths):
    if data['study'] == "PNI": ID_col = "PNI_ID"
    else: ID_col = "MICS_ID"

    map_pths_clean = clean_pths(map_pths, ID_col, method="newest")

In [ ]:
# put surface files into dataframes

In [ ]:
# flip TLEs --> put all lesions on same side

In [ ]:
# get difference maps at 3T (3T ctrl - 3T cases)

In [ ]:
# get difference maps at 7T (7T ctrl - 7T cases)

In [ ]:
# get difference maps of difference maps (3T dif maps - 7T dif maps)